# BG-NBD ve Gamma-Gamma ile CLTV Prediction

###############################################################
### İş Problemi (Business Problem)

FLO satış ve pazarlama faaliyetleri için roadmap belirlemek istemektedir. Şirketin orta uzun vadeli plan yapabilmesi için var olan müşterilerin gelecekte şirkete sağlayacakları potansiyel değerin tahmin edilmesi gerekmektedir.

###############################################################

        Veri Seti Hikayesi

Veri seti son alışverişlerini 2020 - 2021 yıllarında OmniChannel(hem online hem offline alışveriş yapan) olarak yapan müşterilerin geçmiş alışveriş davranışlarından elde edilen bilgilerden oluşmaktadır.


In [113]:
# Değişkenler;

# master_id: Eşsiz müşteri numarası
# order_channel : Alışveriş yapılan platforma ait hangi kanalın kullanıldığı (Android, ios, Desktop, Mobile, Offline)
# last_order_channel : En son alışverişin yapıldığı kanal
# first_order_date : Müşterinin yaptığı ilk alışveriş tarihi
# last_order_date : Müşterinin yaptığı son alışveriş tarihi
# last_order_date_online : Muşterinin online platformda yaptığı son alışveriş tarihi
# last_order_date_offline : Muşterinin offline platformda yaptığı son alışveriş tarihi
# order_num_total_ever_online : Müşterinin online platformda yaptığı toplam alışveriş sayısı
# order_num_total_ever_offline : Müşterinin offline'da yaptığı toplam alışveriş sayısı
# customer_value_total_ever_offline : Müşterinin offline alışverişlerinde ödediği toplam ücret
# customer_value_total_ever_online : Müşterinin online alışverişlerinde ödediği toplam ücret
# interested_in_categories_12 : Müşterinin son 12 ayda alışveriş yaptığı kategorilerin listesi

In [114]:
# To do List:

# GÖREV 1: Veriyi Hazırlama

# GÖREV 2: CLTV Veri Yapısının Oluşturulması

# GÖREV 3: BG/NBD, Gamma-Gamma Modellerinin Kurulması, CLTV'nin hesaplanması

# GÖREV 4: CLTV'ye Göre Segmentlerin Oluşturulması

# GÖREV 5: Tüm süreci fonksiyonlaştırınız.

In [115]:
# Kütüphanelerimizi import edelim;

import pandas as pd
import datetime as dt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from sklearn.preprocessing import MinMaxScaler

# Burada daha önceden hazırlamış olduğumuz bir fonksiyon kütüphanesi de çağırıyoruz;
import et_analysis as et

# Pandas ayarlamalarımızı yapalım;

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.mode.chained_assignment = None

### Görev 1:

In [116]:
# Verimizi okutalım .Dataframe’in kopyasını oluşturalım;

df_ = pd.read_csv("datasets/flo_data_20K.csv")
df = df_.copy()

# Verimize hızlı bir bakış atalım;
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.00,1.00,139.99,799.38,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.00,2.00,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.00,2.00,189.97,395.35,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.00,1.00,39.99,81.98,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.00,1.00,49.99,159.99,[AKTIFSPOR]


In [117]:
# Veri setinin değişkenlerine hızlıca bakalım;

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [118]:
# Betimsel istatistiklerine bakalım;

df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.00,3.11,4.23,1.00,1.00,2.00,4.00,200.00
order_num_total_ever_offline,19945.00,1.91,2.06,1.00,1.00,1.00,2.00,109.00
customer_value_total_ever_offline,19945.00,253.92,301.53,10.00,99.99,179.98,319.97,18119.14
customer_value_total_ever_online,19945.00,497.32,832.60,12.99,149.98,286.46,578.44,45220.13


    Her bir numerik değişkende aykırı değer var olduğunu görüyoruz.

    Bu aşamada aykırı değerleri baskılayalım; 

In [119]:
# Numerik değişkenleri yakalayalım;

num_cols = [col for col in df.columns if df[col].dtypes != "O"]
num_cols

['order_num_total_ever_online',
 'order_num_total_ever_offline',
 'customer_value_total_ever_offline',
 'customer_value_total_ever_online']

In [120]:
# Bir döngü ile kendi hazırladığımız kütüphanemizdeki fonksiyonu çağırarak aykırı değerleri baskılayalım;

for col in num_cols:
    et.replace_with_thresholds(df, col, q1= 0.01, q3= 0.99)

In [121]:
# Betimsel istatistiklere bir kez daha bakalım;

df.describe().T

# max sütunun değiştiğini görüyoruz. Demek ki uç değerleri baskılamışız.

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.00,3.09,3.81,1.00,1.00,2.00,4.00,48.50
order_num_total_ever_offline,19945.00,1.89,1.43,1.00,1.00,1.00,2.00,16.00
customer_value_total_ever_offline,19945.00,251.92,251.02,10.00,99.99,179.98,319.97,3019.88
customer_value_total_ever_online,19945.00,489.71,632.61,12.99,149.98,286.46,578.44,7799.54


In [122]:
# Omnichannel müşterilerin hem online'dan hemde offline platformlardan alışveriş yaptığını biliyoruz.

# Herbir müşterinin toplam alışveriş sayısı ve harcaması için yeni değişkenler oluşturalım;

df["order_num_total"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["customer_value_total"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

# Verimize hızlı bir bakış atalım;;
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,order_num_total,customer_value_total
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.00,1.00,139.99,799.38,[KADIN],5.00,939.37
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.00,2.00,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.00,2013.55
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.00,2.00,189.97,395.35,"[ERKEK, KADIN]",5.00,585.32
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.00,1.00,39.99,81.98,"[AKTIFCOCUK, COCUK]",2.00,121.97
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.00,1.00,49.99,159.99,[AKTIFSPOR],2.00,209.98


In [123]:
# Tarih ifade eden değişkenlerin tipini date'e çevirelim;

date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(pd.to_datetime)

# İşlemimizi kontrol edelim;
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

### Görev 2:

In [124]:
# Veri setindeki en son alışverişin yapıldığı tarihten 2 gün sonrasını analiz tarihi olarak belirleyelim;

print(df["last_order_date"].max())
analysis_date = dt.datetime(2021,6,1)

2021-05-30 00:00:00


In [125]:
# customer_id, recency_cltv_weekly, T_weekly, frequency ve monetary_cltv_avg değerlerinin yer aldığı 
# yeni bir cltv dataframe'i oluşturalım;
cltv_df = pd.DataFrame()

# Atamalarımızı yapalım;
cltv_df["customer_id"] = df["master_id"]

# Aşağıdaki farkı ".astype('timedelta64[D]')" ile dönüştürmemiz gerekir. 
# Yoksa değişken türü zaman datatype'ı olarak kalır.
cltv_df["recency_cltv_weekly"] = ((df["last_order_date"]- df["first_order_date"])) / 7
cltv_df["T_weekly"] = ((analysis_date - df["first_order_date"]))/7
cltv_df["frequency"] = df["order_num_total"]
cltv_df["monetary_cltv_avg"] = df["customer_value_total"] / df["order_num_total"]

# cltv_df' e hızlı bir bakış atalım;
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17 days 00:00:00,30 days 13:42:51.428571428,5.00,187.87
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209 days 20:34:17.142857142,224 days 20:34:17.142857142,21.00,95.88
2,69b69676-1a40-11ea-941b-000d3a38a36f,52 days 06:51:25.714285714,78 days 20:34:17.142857142,5.00,117.06
3,1854e56c-491f-11eb-806e-000d3a38a36f,1 days 13:42:51.428571428,20 days 20:34:17.142857142,2.00,60.98
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83 days 03:25:42.857142857,95 days 10:17:08.571428571,2.00,104.99


    Yukarıdaki verilerin zaman formatında olması verinin okunabilirliğini ve işlenebilirliğini azaltıyor. Bu sorunun üstesinden geleceğiz.

In [126]:
# Atamalarımızı düzeltelim;
cltv_df["customer_id"] = df["master_id"]

# Aşağıdaki farkı ".astype('timedelta64[D]')" ile dönüştürmemiz gerekir. 
# Yoksa değişken türü zaman datatype'ı olarak kalır.
cltv_df["recency_cltv_weekly"] = ((df["last_order_date"]- df["first_order_date"]).astype('timedelta64[D]')) / 7
cltv_df["T_weekly"] = ((analysis_date - df["first_order_date"]).astype('timedelta64[D]'))/7
cltv_df["frequency"] = df["order_num_total"]
cltv_df["monetary_cltv_avg"] = df["customer_value_total"] / df["order_num_total"]

# cltv_df' e hızlı bir bakış atalım;
cltv_df.head()

# Sorunumuz çözüldü :)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.00,30.57,5.00,187.87
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.86,224.86,21.00,95.88
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.29,78.86,5.00,117.06
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.57,20.86,2.00,60.98
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.14,95.43,2.00,104.99


### Görev 3:

In [127]:
# BG/NBD modelini kuralım;
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df['frequency'],
        cltv_df['recency_cltv_weekly'],
        cltv_df['T_weekly'])

# Modelin çalışmasına yönelik ayrıntılı bilgi için;
# "https://lifetimes.readthedocs.io/en/latest/lifetimes.fitters.html" internet sitesine bakabilirsiniz.

# Yine de hızlıca bahsedecek olursak;
# Bu modele göre gelecekteki satın alma davranışları, zaman içerisindeki kesikli noktalar olarak ele alınarak,
# belirli satın alma ihtimaline sahip bir yenilik-sıklık matrisi ile tahmin edilmektedir.

<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 76.17, b: 0.00, r: 3.66>

In [128]:
# 3 ay içerisinde müşterilerden beklenen satın almaları tahmin edelim 
# ve exp_sales_3_month olarak cltv dataframe'ine ekleyelim;

cltv_df["exp_sales_3_month"] = bgf.predict(4*3,                 # Bir ayda 4 hafta, ve 3 ay olduğu içi 4*3 yaptık
                                       cltv_df['frequency'],
                                       cltv_df['recency_cltv_weekly'],
                                       cltv_df['T_weekly'])

# cltv_df' e hızlı bir bakış atalım;
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.00,30.57,5.00,187.87,0.97
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.86,224.86,21.00,95.88,0.98
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.29,78.86,5.00,117.06,0.67
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.57,20.86,2.00,60.98,0.70
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.14,95.43,2.00,104.99,0.40


    Sıfırıncı indeksteki müşterinin önümüzdeki 3 ay içerisinde 1 adet ürün alması beklenebilir diyebiliriz.

In [129]:
# 6 ay içerisinde müşterilerden beklenen satın almaları tahmin edelim 
# ve exp_sales_6_month olarak cltv dataframe'ine ekleyelim;

cltv_df["exp_sales_6_month"] = bgf.predict(4*6,
                                       cltv_df['frequency'],
                                       cltv_df['recency_cltv_weekly'],
                                       cltv_df['T_weekly'])

# cltv_df' e hızlı bir bakış atalım;
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.00,30.57,5.00,187.87,0.97,1.95
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.86,224.86,21.00,95.88,0.98,1.97
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.29,78.86,5.00,117.06,0.67,1.34
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.57,20.86,2.00,60.98,0.70,1.40
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.14,95.43,2.00,104.99,0.40,0.79


In [130]:
# 3. ve 6.aydaki en çok satın alım gerçekleştirecek 10 kişiyi inceleyelim;

display(cltv_df.sort_values("exp_sales_3_month",ascending=False)[:10])

display(cltv_df.sort_values("exp_sales_6_month",ascending=False)[:10])

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.71,67.29,52.50,164.63,4.70,9.40
15611,4a7e875e-e6ce-11ea-8f44-000d3a38a36f,39.71,40.00,29.00,165.30,3.37,6.75
8328,1902bf80-0035-11eb-8341-000d3a38a36f,28.86,33.29,25.00,97.44,3.14,6.28
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.57,58.71,31.00,228.53,3.08,6.17
14373,f00ad516-c4f4-11ea-98f7-000d3a38a36f,38.00,46.43,27.00,141.35,3.00,6.00
10489,7af5cd16-b100-11e9-9757-000d3a38a36f,103.14,111.86,43.00,157.11,2.98,5.96
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,133.14,147.14,49.50,160.20,2.86,5.71
6756,27310582-6362-11ea-a6dc-000d3a38a36f,62.71,64.14,29.00,168.88,2.79,5.59
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,9.71,13.00,17.00,259.87,2.78,5.56
10536,e143b6fa-d6f8-11e9-93bc-000d3a38a36f,104.57,113.43,40.00,176.20,2.76,5.53


,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.71,67.29,52.50,164.63,4.70,9.40
15611,4a7e875e-e6ce-11ea-8f44-000d3a38a36f,39.71,40.00,29.00,165.30,3.37,6.75
8328,1902bf80-0035-11eb-8341-000d3a38a36f,28.86,33.29,25.00,97.44,3.14,6.28
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.57,58.71,31.00,228.53,3.08,6.17
14373,f00ad516-c4f4-11ea-98f7-000d3a38a36f,38.00,46.43,27.00,141.35,3.00,6.00
10489,7af5cd16-b100-11e9-9757-000d3a38a36f,103.14,111.86,43.00,157.11,2.98,5.96
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,133.14,147.14,49.50,160.20,2.86,5.71
6756,27310582-6362-11ea-a6dc-000d3a38a36f,62.71,64.14,29.00,168.88,2.79,5.59
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,9.71,13.00,17.00,259.87,2.78,5.56
10536,e143b6fa-d6f8-11e9-93bc-000d3a38a36f,104.57,113.43,40.00,176.20,2.76,5.53


In [131]:
# Daha hızlı bakmak istersek;

print(cltv_df.sort_values("exp_sales_3_month",ascending=False)[:10].index)
print(cltv_df.sort_values("exp_sales_6_month",ascending=False)[:10].index)

# Bunu kod ile doğrulamak istersek;
print([True if cltv_df.sort_values("exp_sales_3_month",ascending=False)[:10].index[i] == \
               cltv_df.sort_values("exp_sales_6_month",ascending=False)[:10].index[i] else False for i in range(10) ])

Int64Index([7330, 15611, 8328, 19538, 14373, 10489, 4315, 6756, 6666, 10536], dtype='int64')
Int64Index([7330, 15611, 8328, 19538, 14373, 10489, 4315, 6756, 6666, 10536], dtype='int64')
[True, True, True, True, True, True, True, True, True, True]


In [132]:
# Gamma-Gamma modelinin çalışması için "frequency" değişkeninin integer olması gerekmektedir.
print(cltv_df.info())

cltv_df['frequency'] = cltv_df['frequency'].astype(int)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   customer_id          19945 non-null  object 
 1   recency_cltv_weekly  19945 non-null  float64
 2   T_weekly             19945 non-null  float64
 3   frequency            19945 non-null  float64
 4   monetary_cltv_avg    19945 non-null  float64
 5   exp_sales_3_month    19945 non-null  float64
 6   exp_sales_6_month    19945 non-null  float64
dtypes: float64(6), object(1)
memory usage: 1.1+ MB
None


In [133]:
# Kontrol edelim;
cltv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   customer_id          19945 non-null  object 
 1   recency_cltv_weekly  19945 non-null  float64
 2   T_weekly             19945 non-null  float64
 3   frequency            19945 non-null  int64  
 4   monetary_cltv_avg    19945 non-null  float64
 5   exp_sales_3_month    19945 non-null  float64
 6   exp_sales_6_month    19945 non-null  float64
dtypes: float64(5), int64(1), object(1)
memory usage: 1.1+ MB


In [134]:
# Gamma-Gamma modelini fit edelim. 
# Müşterilerin ortalama bırakacakları değeri tahminleyip exp_average_value olarak cltv dataframe'ine ekleyelim;

ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])
cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                cltv_df['monetary_cltv_avg'])

# cltv_df' e hızlı bir bakış atalım;
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.00,30.57,5,187.87,0.97,1.95,193.63
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.86,224.86,21,95.88,0.98,1.97,96.67
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.29,78.86,5,117.06,0.67,1.34,120.97
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.57,20.86,2,60.98,0.70,1.40,67.32
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.14,95.43,2,104.99,0.40,0.79,114.33


In [135]:
# 6 aylık CLTV hesaplayalım ve cltv ismiyle dataframe'e ekleyelim;
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency_cltv_weekly'],
                                   cltv_df['T_weekly'],
                                   cltv_df['monetary_cltv_avg'],
                                   time=6,
                                   freq="W",
                                   discount_rate=0.01)

cltv_df["cltv"] = cltv

# cltv_df' e hızlı bir bakış atalım;
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.00,30.57,5,187.87,0.97,1.95,193.63,395.73
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.86,224.86,21,95.88,0.98,1.97,96.67,199.43
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.29,78.86,5,117.06,0.67,1.34,120.97,170.22
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.57,20.86,2,60.98,0.70,1.40,67.32,98.95
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.14,95.43,2,104.99,0.40,0.79,114.33,95.01


In [137]:
# CLTV değeri en yüksek 10 kişiyi gözlemleyelim;
cltv_df.sort_values("cltv",ascending=False)[:10]

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,2.86,7.86,4,1401.77,1.09,2.19,1449.03,3327.71
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,6.14,13.14,11,758.04,1.97,3.94,767.32,3172.22
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,51.71,101.00,7,1106.47,0.72,1.44,1127.61,1708.98
12438,625f40a2-5bd2-11ea-98b0-000d3a38a36f,74.29,74.57,16,501.85,1.57,3.13,506.14,1662.52
8868,9ce6e520-89b0-11ea-a6e7-000d3a38a36f,3.43,34.43,8,601.23,1.27,2.53,611.49,1623.81
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.71,67.29,52,164.63,4.70,9.40,165.12,1613.30
6402,851de3b4-8f0c-11eb-8cb8-000d3a38a36f,8.29,9.43,2,862.69,0.79,1.59,923.68,1538.86
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,9.71,13.00,17,259.87,2.78,5.56,262.07,1529.23
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.57,58.71,31,228.53,3.08,6.17,229.61,1485.82
14858,031b2954-6d28-11eb-99c4-000d3a38a36f,14.86,15.57,3,743.59,0.87,1.74,778.05,1423.00


### GÖREV 4:

In [138]:
# 6 aylık standartlaştırılmış CLTV'ye göre tüm müşterileri 4 gruba (segmente) ayıralım,
# Grup isimlerini veri setine ekleyelim ve cltv_segment ismi ile atamasını yapalım;

cltv_df["cltv_segment"] = pd.qcut(cltv_df["cltv"], 4, labels=["D", "C", "B", "A"])

# cltv_df' e hızlı bir bakış atalım;
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv,cltv_segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.00,30.57,5,187.87,0.97,1.95,193.63,395.73,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.86,224.86,21,95.88,0.98,1.97,96.67,199.43,B
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.29,78.86,5,117.06,0.67,1.34,120.97,170.22,B
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.57,20.86,2,60.98,0.70,1.40,67.32,98.95,D
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.14,95.43,2,104.99,0.40,0.79,114.33,95.01,D


# Görev 5:

In [141]:
def create_cltv_df(dataframe):

    # Veriyi Hazırlama
    columns = [col for col in df.columns if df[col].dtypes != "O"]
    
    for col in columns:
        et.replace_with_thresholds(dataframe, col, q1=0.01, q3=0.99)

    dataframe["order_num_total"] = dataframe["order_num_total_ever_online"] + dataframe["order_num_total_ever_offline"]
    dataframe["customer_value_total"] = dataframe["customer_value_total_ever_offline"] + \
        dataframe["customer_value_total_ever_online"]
    
    dataframe = dataframe[~(dataframe["customer_value_total"] == 0) | (dataframe["order_num_total"] == 0)]
    date_columns = dataframe.columns[dataframe.columns.str.contains("date")]
    dataframe[date_columns] = dataframe[date_columns].apply(pd.to_datetime)

    # CLTV veri yapısının oluşturulması
    dataframe["last_order_date"].max()  # 2021-05-30
    analysis_date = dt.datetime(2021, 6, 1)
    cltv_df = pd.DataFrame()
    cltv_df["customer_id"] = dataframe["master_id"]
    cltv_df["recency_cltv_weekly"] = ((dataframe["last_order_date"] - dataframe["first_order_date"]).   \
                                      astype('timedelta64[D]')) / 7
    cltv_df["T_weekly"] = ((analysis_date - dataframe["first_order_date"]).astype('timedelta64[D]')) / 7
    cltv_df["frequency"] = dataframe["order_num_total"]
    cltv_df["monetary_cltv_avg"] = dataframe["customer_value_total"] / dataframe["order_num_total"]
    cltv_df = cltv_df[(cltv_df['frequency'] > 1)]

    # BG-NBD Modelinin Kurulması
    bgf = BetaGeoFitter(penalizer_coef=0.001)
    bgf.fit(cltv_df['frequency'],
            cltv_df['recency_cltv_weekly'],
            cltv_df['T_weekly'])
    cltv_df["exp_sales_3_month"] = bgf.predict(4 * 3,
                                               cltv_df['frequency'],
                                               cltv_df['recency_cltv_weekly'],
                                               cltv_df['T_weekly'])
    cltv_df["exp_sales_6_month"] = bgf.predict(4 * 6,
                                               cltv_df['frequency'],
                                               cltv_df['recency_cltv_weekly'],
                                               cltv_df['T_weekly'])

    # Gamma-Gamma Modelinin Kurulması
    cltv_df['frequency'] = cltv_df['frequency'].astype(int)
    ggf = GammaGammaFitter(penalizer_coef=0.01)
    ggf.fit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])
    cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                           cltv_df['monetary_cltv_avg'])

    # Cltv tahmini
    cltv = ggf.customer_lifetime_value(bgf,
                                       cltv_df['frequency'],
                                       cltv_df['recency_cltv_weekly'],
                                       cltv_df['T_weekly'],
                                       cltv_df['monetary_cltv_avg'],
                                       time=6,
                                       freq="W",
                                       discount_rate=0.01)
    cltv_df["cltv"] = cltv

    # CLTV segmentleme
    cltv_df["cltv_segment"] = pd.qcut(cltv_df["cltv"], 4, labels=["D", "C", "B", "A"])

    return cltv_df

In [142]:
# Fonksiyonumuzu deneyelim;

cltv_df = create_cltv_df(df)

# İlk 10 gözlemi görelim
cltv_df.head(10)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv,cltv_segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.00,30.57,5,187.87,0.97,1.95,193.63,395.73,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.86,224.86,21,95.88,0.98,1.97,96.67,199.43,B
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.29,78.86,5,117.06,0.67,1.34,120.97,170.22,B
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.57,20.86,2,60.98,0.70,1.40,67.32,98.95,D
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.14,95.43,2,104.99,0.40,0.79,114.33,95.01,D
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,120.86,132.29,3,66.95,0.38,0.77,71.35,57.43,D
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,32.57,64.86,4,93.98,0.65,1.30,98.13,134.28,C
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,12.71,54.57,2,81.81,0.52,1.04,89.57,97.70,D
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,58.43,70.71,5,210.94,0.71,1.42,217.30,322.73,A
9,1143f032-440d-11ea-8b43-000d3a38a36f,61.71,96.00,2,82.98,0.39,0.79,90.81,75.22,D
